Ufuk Altun

using Pkg; pkg"add CUDA Knet IterTools FileIO JLD2 Images Plots PyCall"

In [1]:
using PyCall
@pyimport torch

## Use Pycall to imitate dataloading of the original code

In [10]:
using PyCall

py"""
import argparse
import os
import random
import torch
import numpy as np
from torchvision import transforms
from PIL import Image, ImageFile
#import datasets

parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', type=str, default="/Users/ufukaltun/PycharmProjects/DGVGS/DGvGS-main/data/pre")
parser.add_argument('--dataset', type=str, default='PACS')
parser.add_argument('--method', type=str, default='deep-all')
parser.add_argument('--seed', type=int, default=0)
parser.add_argument('--iterations', type=int, default=300)
parser.add_argument('--val_every', type=int, default=20)
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--lr', type=float, default=1e-5)
parser.add_argument('--weight_decay', type=float, default=5e-5)
parser.add_argument('--test_dom_idx', type=int, default=1)
parser.add_argument('--output_dir', type=str, default="result/train")
args = parser.parse_args()
os.makedirs(args.output_dir, exist_ok=True)
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'


ImageFile.LOAD_TRUNCATED_IMAGES = True


def get_domains(dataset_name):
    return globals()[dataset_name].DOMAINS


class ImageDataset(torch.utils.data.Dataset):

    def __init__(self, file_path, image_dir, transform=None):
        self.file_path = file_path
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self._read_file()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = os.path.join(self.image_dir, self.image_paths[idx])

        with open(path, 'rb') as f:
            image = Image.open(f).convert('RGB')
        label = self.labels[idx]

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def _read_file(self):
        with open(self.file_path) as f:
            for line in f:
                path, label = line.strip().split(',')
                self.image_paths.append(path)
                self.labels.append(int(label) - 1)

class MultiDomainDataset:

    def __init__(self, root_dir, test_dom_idx):
        images_dir = os.path.join(root_dir, 'images')
        split_dir = os.path.join(root_dir, 'split')

        domains = [f.name for f in os.scandir(images_dir) if f.is_dir()]
        domains.sort()

        test_dom = domains[test_dom_idx]
        train_doms = [d for d in domains if d != test_dom]

        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])])

        train_datasets, val_datasets = [], []
        for dom_name in train_doms:
            train_datasets.append(ImageDataset(
                os.path.join(split_dir, dom_name + '_train.txt'),
                images_dir,
                transform))
            val_datasets.append(ImageDataset(
                os.path.join(split_dir, dom_name + '_val.txt'),
                images_dir,
                transform))

        self.datasets = {}
        self.datasets['train'] = train_datasets
        self.datasets['val'] = torch.utils.data.ConcatDataset(val_datasets)
        self.datasets['test'] = ImageDataset(
            os.path.join(split_dir, test_dom + '_test.txt'),
            images_dir,
            transform)

    def __getitem__(self, phase):
        if phase in ['train', 'val', 'test']:
            return self.datasets[phase]
        else:
            raise ValueError


class VLCS(MultiDomainDataset):

    N_CLASSES = 5
    DOMAINS = ['C', 'L', 'S', 'V']

    def __init__(self, root_dir, test_dom_idx):
        self.root_dir = os.path.join(root_dir, 'VLCS/')
        super().__init__(self.root_dir, test_dom_idx)


class PACS(MultiDomainDataset):

    N_CLASSES = 7
    DOMAINS = ['A', 'C', 'P', 'S']

    def __init__(self, root_dir, test_dom_idx):
        self.root_dir = os.path.join(root_dir, 'PACS/')
        super().__init__(self.root_dir, test_dom_idx)

############# data loader ###########

class InfiniteDataLoader(torch.utils.data.DataLoader):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dataset_iterator = super().__iter__()

    def __iter__(self):
        return self

    def __next__(self):
        try:
            batch = next(self.dataset_iterator)
        except StopIteration:
            self.dataset_iterator = super().__iter__()
            batch = next(self.dataset_iterator)
        return batch

dataset = PACS(args.data_dir, args.test_dom_idx)


############# Model ##########

def get_model(device, dataset, args):
    if args.method == 'deep-all':
        return ModelDA(device, dataset, args)
    elif args.method in ['agr-sum', 'agr-rand', 'pcgrad']:
        return ModelGS(device, dataset, args)
    else:
        raise ValueError


class ModelDA:

    def __init__(self, device, dataset, args):
        self.device = device
        self.args = args
        self._create_dataloaders(dataset, args)

    def _create_dataloaders(self, dataset, args):

        def get_dataloader(dataset, batch_size, is_train=False):
            if is_train:
                return InfiniteDataLoader(
                    dataset=dataset,
                    batch_size=batch_size,
                    shuffle=True,
                    drop_last=True)
            else:
                return torch.utils.data.DataLoader(
                    dataset=dataset,
                    batch_size=batch_size,
                    shuffle=False,
                    drop_last=False)

        self.train_loaders = []
        for dom_dataset in dataset['train']:
            self.train_loaders.append(get_dataloader(dom_dataset, args.batch_size, True))
        self.val_loader = get_dataloader(dataset['val'], args.batch_size)
        self.test_loader = get_dataloader(dataset['test'], args.batch_size)

    def _prepare_batch(self, batch):
        inputs, targets = batch
        inputs = inputs.to(self.device)
        targets = targets.to(self.device)
        return inputs, targets

    def train(self):
        train_iterator = zip(*self.train_loaders)
        val_iterator = self.val_loader
        test_iterator = self.test_loader
        
        iterations = self.args.iterations
        val_every = self.args.val_every

        return train_iterator, val_iterator, test_iterator
        #for it in range(iterations):
        # Training
        #    train_batches = [self._prepare_batch(batch) for batch in next(train_iterator)]
        #    train_loss, train_acc = self._train_step(train_batches)
        #    is_train = True
        #    inputs = torch.cat([x for x, _ in train_batches])
        #    targets = torch.cat([y for _, y in train_batches])



###################

#dataset = vars(datasets)[args.dataset](args.data_dir, args.test_dom_idx)
#dataset = PACS(args.data_dir, args.test_dom_idx)
model = ModelDA(device, dataset, args)
train_iterator, val_iterator, test_iterator = model.train()

def prep_batch(batch):
    inputs, targets = batch
    inputs = inputs.to('cpu')
    targets = targets.to('cpu')
    return inputs, targets
    

############## def return data #######
def return_data(train_iterator):
    train_batches = [prep_batch(batch) for batch in next(train_iterator)]
    inputs = torch.cat([x for x, _ in train_batches])
    targets = torch.cat([y for _, y in train_batches])
    return inputs, targets

def return_val_data(val_iterator):
    train_batches = [prep_batch(batch) for batch in val_iterator]
    inputs = torch.cat([x for x, _ in train_batches])
    targets = torch.cat([y for _, y in train_batches])
    return inputs, targets
    
def return_data_gs(train_iterator):
    train_batches = [prep_batch(batch) for batch in next(train_iterator)]
    return train_batches
    
"""   

In [3]:
include("models.jl");
include("training.jl");

#### Generate an alexnet model

In [11]:
model = generate_alexnet_model( 7 ; pretrained = true);

#### Set optimizers of each parameter to Adam. Parameter sizes should be:
##### Original Alexnet
- (11, 11, 3, 64)
- (1, 1, 64, 1)
- (5, 5, 64, 192)
- (1, 1, 192, 1)
- (3, 3, 192, 384)
- (1, 1, 384, 1)
- (3, 3, 384, 256)
- (1, 1, 256, 1)
- (3, 3, 256, 256)
- (1, 1, 256, 1)
- (4096, 9216)
- (4096,)
- (4096, 4096)
- (4096,)
- (7, 4096)
- (7,)

In [12]:
for param in params(model)
    param.opt = Adam(; lr=1e-5)
    println(size(param))
end

(11, 11, 3, 64)
(1, 1, 64, 1)
(5, 5, 64, 192)
(1, 1, 192, 1)
(3, 3, 192, 384)
(1, 1, 384, 1)
(3, 3, 384, 256)
(1, 1, 256, 1)
(3, 3, 256, 256)
(1, 1, 256, 1)
(4096, 9216)
(4096,)
(4096, 4096)
(4096,)
(7, 4096)
(7,)


#### Define batchsize, model path and target domain  

In [13]:
num_iter=600
check_freq=20
batchsize = 128
atype = Array{Float32}
data_path = "/Users/ufukaltun/Documents/koç/dersler/ku deep learning/project/data";
target = "cartoon"

"cartoon"

In [14]:
inputs_val, targets_val = py"return_val_data(val_iterator)"
inputs_val = permutedims(inputs_val.detach().numpy(), (4, 3, 2, 1));
targets_val = targets_val.detach().numpy();

In [15]:
inputs_test, targets_test = py"return_val_data(test_iterator)"
inputs_test = permutedims(inputs_test.detach().numpy(), (4, 3, 2, 1));
targets_test = targets_test.detach().numpy();

## Training of Baseline model with "Cartoon" target, PACS dataset

In [9]:
n=1
results=[]
best_acc=-1
for it in progress(1:num_iter)
    inputs_train, targets_train = py"return_data(train_iterator)"
    inputs_train = permutedims(inputs_train.detach().numpy(), (4, 3, 2, 1));
    targets_train = targets_train.detach().numpy();

    D = @diff model(inputs_train, targets_train);
    for w in Knet.params(model)
        g = grad(D, w)
        update!(w, g)
    end
    if n==1 
        println("First batch is trained successfully")
        best_acc = accuracy(model(inputs_val),targets_val)
    end
    if n%check_freq == 0
        acc = accuracy(model(inputs_val),targets_val)
        if acc > best_acc
            best_acc = acc
            save(string("best_model_b_debugging_target_", target,".jld2"),"weights",model)
        end
        result=(model(inputs_train, targets_train), model(inputs_val, targets_val), 
                    accuracy(model(inputs_train),targets_train), acc)
        println("Train loss: ",result[1],"  Val loss: ",result[2],
                "  Train acc: ",result[3],"  Val acc: ",result[4])
        push!(results,result)
    end

    (n += 1) > num_iter && break
end 
println("Training ended successfully, saving the results")
save(string("results_b_debugging_target_", target,".jld2"),"results",results)
best_model = load(string("best_model_b_debugging_target_", target,".jld2"))["weights"]
tst_loss = best_model(inputs_test, targets_test)
tst_acc = accuracy(model(inputs_test),targets_test)
println("Test loss: ", tst_loss, "  Test acc: ", tst_acc)

┣                    ┫ [0.17%, 1/600, 00:00/00:01, 995.64i/s] 

First batch is trained successfully


┣▋                   ┫ [3.33%, 20/600, 12:01/06:00:43, 34.64s/i] 

Train loss: 1.4085758

┣▋                   ┫ [3.50%, 21/600, 13:51/06:35:45, 109.64s/i] 

...

Train loss: 0.007621357  Val loss: 0.34247118  Train acc: 1.0  Val acc: 0.8922345483359746


┣███████████████████▎┫ [96.83%, 581/600, 06:23:57/06:36:31, 101.88s/i] 

Train loss: 0.012343519  Val loss: 0.33897206  Train acc: 1.0  Val acc: 0.8969889064976229


┣████████████████████┫ [100.00%, 600/600, 06:35:24/06:35:24, 35.26s/i] 

Train loss: 0.013096458  Val loss: 0.34188887  Train acc: 1.0  Val acc: 0.8858954041204438
Training ended successfully, saving the results
Test loss: 1.3054203  Test acc: 0.6675191815856778


## Training of Gradient Surgery model with "Cartoon" target, PACS dataset

In [16]:
n=1
results=[]
best_acc=0
for it in progress(1:num_iter)
    train_batches = py"return_data_gs(train_iterator)"
    inputs1, targets1 = train_batches[1];
    inputs1 = permutedims(inputs1.detach().numpy(), (4, 3, 2, 1));
    targets1 = targets1.detach().numpy();
    
    inputs2, targets2 = train_batches[2];
    inputs2 = permutedims(inputs2.detach().numpy(), (4, 3, 2, 1));
    targets2 = targets2.detach().numpy();
    
    inputs3, targets3 = train_batches[3];
    inputs3 = permutedims(inputs3.detach().numpy(), (4, 3, 2, 1));
    targets3 = targets3.detach().numpy();
    
    D1 = @diff model(inputs1, targets1)
    D2 = @diff model(inputs2, targets2)
    D3 = @diff model(inputs3, targets3)

    for w in Knet.params(model)
        g1 = grad(D1, w)
        g2 = grad(D2, w)
        g3 = grad(D3, w)
        MaskGS = abs.(sign.(g1).+sign.(g2).+sign.(g3)).==3
        g = (g1.+g2.+g3).*MaskGS
        update!(w, g)
    end

    if n==1 
        println("First batch is trained successfully")
        best_acc = accuracy(model(inputs_val),targets_val)
    end

    if n%check_freq == 0
        acc = accuracy(model(inputs_val),targets_val)
        if acc > best_acc
            best_acc = acc
            save(string("best_model_gs_debugging_target_", target,".jld2"),"weights",model)
        end
        temp_loss = (model(inputs1, targets1) + model(inputs2, targets2) + model(inputs3, targets3))/3
        temp_acc = (accuracy(model(inputs1),targets1) + accuracy(model(inputs2),targets2) + accuracy(model(inputs3),targets3))/3
        result=(temp_loss, model(inputs_val, targets_val), temp_acc, acc)
        println("Train loss: ",result[1],"  Val loss: ",result[2],
                "  Train acc: ",result[3],"  Val acc: ",result[4])
        push!(results,result)
    end

    (n += 1) > num_iter && break
end 
println("Training ended successfully, saving the results")
save(string("results_gs_debugging_target_", target,".jld2"),"results",results)
best_model = load(string("best_model_gs_debugging_target_", target,".jld2"))["weights"]
tst_loss = best_model(inputs_test, targets_test)
tst_acc = accuracy(model(inputs_test),targets_test)
println("Test loss: ", tst_loss, "  Test acc: ", tst_acc)

┣                    ┫ [0.17%, 1/600, 00:00/00:00, 1839.36i/s] 

First batch is trained successfully


┣▋                   ┫ [3.33%, 20/600, 11:48/05:53:52, 34.37s/i] 

Train loss: 2.1176128

┣▋                   ┫ [3.50%, 21/600, 13:27/06:24:26, 99.56s/i] 

...

Train loss: 0.14438382  Val loss: 0.3924485  Train acc: 0.9553701015965167  Val acc: 0.8652931854199684


┣███████████████████▎┫ [96.83%, 581/600, 06:21:50/06:34:19, 97.60s/i] 

Train loss: 0.12688617  Val loss: 0.3879612  Train acc: 0.9469829598506069  Val acc: 0.8621236133122029


┣████████████████████┫ [100.00%, 600/600, 06:33:24/06:33:24, 37.68s/i] 

Train loss: 0.12101819  Val loss: 0.38308683  Train acc: 0.9436403289558629  Val acc: 0.8763866877971473
Training ended successfully, saving the results
Test loss: 1.0162643  Test acc: 0.7135549872122762
